<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [44]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

������� ������� ��������: 1251


"wget" �� �������� ���������� ��� �������
��������, ����������� ���������� ��� �������� ������.


In [45]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("D:/SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [46]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [47]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [48]:
# method most_simmilar
my_dog = 'dog'
my_cat = 'cat'
top = 5
count = 0
idxs = []
for word in wv_embeddings.most_similar(positive=[my_dog], topn = top):
    if my_cat == word[0]:
        print(f'Слово {my_cat} входит в топ-{top} к слову {my_dog} и находится на {wv_embeddings.most_similar(positive=[my_dog], topn=top).index(word) + 1} месте')
        break
    elif my_cat in word[0]:
        count += 1
        idxs.append(wv_embeddings.most_similar(positive=[my_dog], topn = top).index(word) + 1)
    else:
        count += 1
if count == top:
    print(f'Слово "{my_cat}" НЕ входит в топ-{top} близких к слову "{my_dog}"')
if len(idxs) != 0:
    print(f'Однако, на {idxs[0]} позиции в топ-{top} находится слово \
"{wv_embeddings.most_similar(positive=[my_dog], topn = top)[idxs[0]-1][0]}"')

Слово "cat" НЕ входит в топ-5 близких к слову "dog"
Однако, на 4 позиции в топ-5 находится слово "cats"


***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [49]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [50]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    embed_list = []
    for word in tokenizer.tokenize(question.lower()):
        try:
            embed_list.append(embeddings[word])
        except (KeyError):
            continue
    return np.mean(embed_list, axis=0) if len(embed_list) > 0 else np.zeros(dim, dtype='float32')

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [51]:
# Предложение
question = "I love neural networks"
tokenizer = MyTokenizer()
round(question_to_vec(question, wv_embeddings, tokenizer)[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [52]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    hits_value = sum(1 for rank in dup_ranks if rank <= k)
    return hits_value

In [53]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0
Hits@4 = 1


In [54]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Вычисляем сумму для всех релевантных дубликатов
    dcg_value = 0.0
    # Делим на общее количество вопросов
    for rank in dup_ranks:
        if rank <= k:
            dcg_value += 1.0 / math.log2(rank + 1)
    return dcg_value

In [55]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [56]:
import pandas as pd

In [57]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [1]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [1, 1, 1, 1]
Ваш ответ DCG: [1.0, 1.0, 1.0, 1.0]


У вас должно получиться

In [58]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [59]:
!unzip stackoverflow_similar_questions.zip

������� ������� ��������: 1251


"unzip" �� �������� ���������� ��� �������
��������, ����������� ���������� ��� �������� ������.


Считайте данные.

In [60]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [61]:
validation_data = read_corpus('stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [62]:
len(validation_data)

3760

Размер нескольких первых строк

In [63]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [65]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    question_tokens = tokenizer.tokenize(question)
    question_embedding = np.zeros(dim)

    for token in question_tokens:
        if token in embeddings:
            question_embedding += embeddings[token]

    if question_tokens:
        question_tokens /= len(question_tokens)
    
    candidate_similarities = []
    for idx, candidate in enumerate(candidates):
        candidate_tokens = tokenizer.tokenize(candidate)
        candidate_embedding = np.zeros(dim)

        for token in candidate_tokens:
            if token in embeddings:
                candidate_embedding += embeddings[token]
        
        if candidate_tokens:
            candidate_embedding /= len(candidate_tokens)

        if np.linalg.norm(question_embedding) == 0 or np.linalg.norm(candidate_embedding) == 0:
            similarity = 0
        else:
            similarity = np.dot(question_embedding, candidate_embedding) / (np.linalg.norm(question_embedding) * np.linalg.norm(candidate_embedding))

        candidate_similarities.append((idx, candidate, similarity))
    candidate_similarities.sort(key=lambda x: x[2], reverse=True)
    result = [(item[0], item[1]) for item in candidate_similarities]
    return result

In [66]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
    Ранжирует кандидатов по семантической близости к вопросу с использованием cosine_similarity
    
    Параметры:
        question: строка с вопросом
        candidates: список строк-кандидатов [a, b, c]
        embeddings: словарь с эмбеддингами слов
        tokenizer: токенизатор для разбиения текста на слова
        dim: размерность векторов эмбеддингов (по умолчанию 200)
    
    Возвращает:
        Список кортежей (исходный индекс, кандидат, оценка сходства), отсортированных по убыванию близости
        Например: [(2, c, 0.95), (0, a, 0.8), (1, b, 0.6)]
    """
    # Создаем копию embeddings, чтобы не изменять исходный словарь
    embeddings = deepcopy(embeddings)
    
    # 1. Получаем эмбеддинг вопроса
    question_tokens = tokenizer.tokenize(question)
    question_embedding = np.zeros(dim)
    
    for token in question_tokens:
        if token in embeddings:
            question_embedding += embeddings[token]
    
    if question_tokens:
        question_embedding /= len(question_tokens)
    
    # 2. Обрабатываем каждого кандидата
    results = []
    for idx, candidate in enumerate(candidates):
        candidate_tokens = tokenizer.tokenize(candidate)
        candidate_embedding = np.zeros(dim)
        
        for token in candidate_tokens:
            if token in embeddings:
                candidate_embedding += embeddings[token]
        
        if candidate_tokens:
            candidate_embedding /= len(candidate_tokens)
        
        # 3. Вычисляем косинусную близость с помощью sklearn
        similarity = cosine_similarity(
            question_embedding.reshape(1, -1),
            candidate_embedding.reshape(1, -1)
        )[0][0]
        
        results.append((idx, candidate, similarity))
    
    # 4. Сортируем по убыванию сходства
    results.sort(key=lambda x: x[2], reverse=True)
    
    return results

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [67]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [68]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("D:/SO_vectors_200.bin", binary=True)
tokenizer = MyTokenizer()

In [69]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it', 0.5462909752319829), (0, 'Convert Google results object (pure js) to Python object', 0.47886280839497875), (2, 'How to use jQuery AJAX for an outside domain?', 0.16188453100360892)]

[(1, 'WPF- How to update the changes in list item of a list', 0.4768912406597637), (0, 'Getting all list items of an unordered list in PHP', 0.4569547639204964), (2, 'select2 not displaying search results', 0.1632903503238101)]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [70]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(None, 'WPF- How to update the changes in list item of a list'),
            (None, 'Getting all list items of an unordered list in PHP'), #скрыт
            (None, 'select2 not displaying search results') #скрыт
         ]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


(1, 'WPF- How to update the changes in list item of a list', 0.4768912406597637), (0, 'Getting all list items of an unordered list in PHP', 0.4569547639204964), (2, 'select2 not displaying search results', 0.1632903503238101)

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [71]:
from tqdm.notebook import tqdm

In [80]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def rank_candidates_optimized(question, candidates, embeddings, tokenizer, dim=200):
    """
    Оптимизированная версия: векторизованные операции, меньше циклов.
    """
    # Эмбеддинг вопроса (среднее по токенам)
    q_tokens = tokenizer.tokenize(question)
    q_embed = np.mean([embeddings[token] for token in q_tokens if token in embeddings], axis=0)
    if len(q_tokens) == 0 or np.all(q_embed == 0):
        q_embed = np.zeros(dim)  # если токенов нет, возвращаем нулевой вектор

    # Эмбеддинги кандидатов (разом, без вложенных циклов)
    candidates_embeds = []
    for candidate in candidates:
        c_tokens = tokenizer.tokenize(candidate)
        c_embed = np.mean([embeddings[token] for token in c_tokens if token in embeddings], axis=0)
        if len(c_tokens) == 0 or np.all(c_embed == 0):
            c_embed = np.zeros(dim)
        candidates_embeds.append(c_embed)
    
    # Косинусная близость для всех кандидатов сразу
    sims = cosine_similarity([q_embed], candidates_embeds)[0]
    
    # Сортировка по убыванию сходства
    ranked_indices = np.argsort(sims)[::-1]
    return [(idx, candidates[idx], sims[idx]) for idx in ranked_indices]

In [83]:
def rank_candidates_fixed(question, candidates, embeddings, tokenizer, dim=200):
    # Эмбеддинг вопроса
    q_tokens = tokenizer.tokenize(question)
    q_embed = np.zeros(dim)
    valid_tokens = 0
    
    for token in q_tokens:
        if token in embeddings:
            vec = embeddings[token]
            if isinstance(vec, np.ndarray) and vec.shape == (dim,):
                q_embed += vec
                valid_tokens += 1
    
    if valid_tokens > 0:
        q_embed /= valid_tokens

    # Эмбеддинги кандидатов
    candidates_embeds = []
    for candidate in candidates:
        c_tokens = tokenizer.tokenize(candidate)
        c_embed = np.zeros(dim)
        valid_ctokens = 0
        
        for token in c_tokens:
            if token in embeddings:
                vec = embeddings[token]
                if isinstance(vec, np.ndarray) and vec.shape == (dim,):
                    c_embed += vec
                    valid_ctokens += 1
        
        if valid_ctokens > 0:
            c_embed /= valid_ctokens
        candidates_embeds.append(c_embed)

    # Косинусная близость (с проверкой на нулевые векторы)
    if np.all(q_embed == 0):
        return [(i, cand, 0.0) for i, cand in enumerate(candidates)]
    
    sims = cosine_similarity([q_embed], candidates_embeds)[0]
    ranked = sorted(enumerate(zip(candidates, sims)), key=lambda x: x[1][1], reverse=True)
    return [(idx, cand, score) for idx, (cand, score) in ranked]

In [85]:
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import logging

logging.basicConfig(filename='errors.log', level=logging.ERROR)

def rank_candidates_fixed(question, candidates, embeddings, tokenizer, dim=200):
    # ... (используйте исправленную версию из предыдущего ответа) ...
    q_tokens = tokenizer.tokenize(question)
    q_embed = np.zeros(dim)
    valid_tokens = 0
    
    for token in q_tokens:
        if token in embeddings:
            vec = embeddings[token]
            if isinstance(vec, np.ndarray) and vec.shape == (dim,):
                q_embed += vec
                valid_tokens += 1
    
    if valid_tokens > 0:
        q_embed /= valid_tokens

    # Эмбеддинги кандидатов
    candidates_embeds = []
    for candidate in candidates:
        c_tokens = tokenizer.tokenize(candidate)
        c_embed = np.zeros(dim)
        valid_ctokens = 0
        
        for token in c_tokens:
            if token in embeddings:
                vec = embeddings[token]
                if isinstance(vec, np.ndarray) and vec.shape == (dim,):
                    c_embed += vec
                    valid_ctokens += 1
        
        if valid_ctokens > 0:
            c_embed /= valid_ctokens
        candidates_embeds.append(c_embed)

    # Косинусная близость (с проверкой на нулевые векторы)
    if np.all(q_embed == 0):
        return [(i, cand, 0.0) for i, cand in enumerate(candidates)]
    
    sims = cosine_similarity([q_embed], candidates_embeds)[0]
    ranked = sorted(enumerate(zip(candidates, sims)), key=lambda x: x[1][1], reverse=True)
    return [(idx, cand, score) for idx, (cand, score) in ranked]

wv_ranking = []
for i, line in enumerate(tqdm(validation_data)):
    try:
        q, *ex = line
        if not q or not ex:
            continue
        ranks = rank_candidates_fixed(q, ex, wv_embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    except Exception as e:
        logging.error(f"Пример {i}: {str(e)}")
        continue

print(f"Обработано примеров: {len(wv_ranking)}")

100%|██████████| 3760/3760 [01:07<00:00, 55.40it/s]

Обработано примеров: 3760


In [87]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 1403.72it/s]

DCG@   1: 1082.000 | Hits@   1: 1082.000
DCG@   5: 1307.494 | Hits@   5: 1511.000
DCG@  10: 1384.233 | Hits@  10: 1748.000
DCG@ 100: 1553.718 | Hits@ 100: 2599.000
DCG@ 500: 1642.380 | Hits@ 500: 3292.000
DCG@1000: 1691.464 | Hits@1000: 3760.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [74]:
train_data = read_corpus('D:/Projects/DeepLearning_2sem/stackoverflow_similar_questions/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [88]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [91]:
# Создаем общий корпус текстов
corpus = []
for line in train_data:
    question, *candidates = line
    if not question or not candidates:
        continue
    question_tokens = tokenizer.tokenize(question.lower())
    for candidate in candidates:
        candidate_tokens = tokenizer.tokenize(candidate.lower())
        combined_tokens = question_tokens + candidate_tokens
        corpus.append(combined_tokens)

In [93]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5,                # Размер окна контекста
    min_count=2,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [94]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

 27%|██▋       | 1000/3760 [06:19<17:27,  2.63it/s]


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Вид токенизации зависит от задачи, но для баланса качества и скорости выбирают лематизацию через spaCy. Если критична производительность, то WordPunctTokenizer

Нормализация помогает, потому что она уменьшает размерность эмбеддингов, повышает семантическую схожесть слов, снижает переобучение

Лучше бы справились эмбединги FastText, GloVe, BERT

Использование классического Word2Vec без дообучения на специфичных данных (технические термины, вопросы/ответы).
Отсутствие обработки OOV-слов (например, "PyTorch" мог отсутствовать в словаре).        
Разделение слов без учёта контекста (например, "Python" и "Python's" считались разными словами).
Игнорирование стоп-слов, которые могут быть важны для вопросов ("what", "how").
Косинусная близость между средними векторами часто хуже учитывает семантику, чем современные методы.